In [2]:
#matplotlib inline
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from dataset.dataloader import *
import operators as ops
import os


In [3]:
class VGG8:
    def __init__(self):
        self.criterion = ops.SoftmaxWithLoss()
        self.conv_layers = [
            # Layer 1 (B, 1, 28, 28) -> (B, 32, 28, 28)
            ops.Conv2D(in_channels=1, out_channels=32, kernel_size=3, padding=1, stride=1, name="L1_C"),
            ops.ReLU(name="L1_R"),

            # Layer2 (B, 32, 28, 28) -> (B, 64, 14, 14)
            ops.Conv2D(in_channels=32, out_channels=64, kernel_size=3, padding=1, stride=1, name="L2_C"),
            ops.ReLU(name="L2_R"),
            ops.MaxPooling(kernel_size=2, stride=2, name="L2_M"),
            
            # Layer 3 (B, 64, 14, 14) -> (B, 64, 14, 14)
            ops.Conv2D(in_channels=64, out_channels=64, kernel_size=3, padding=1, stride=1, name="L3_C"),
            ops.ReLU(name="L3_R"),

            # Layer 4 (B, 64, 14, 14) -> (B, 128, 7, 7)
            ops.Conv2D(in_channels=64, out_channels=128, kernel_size=3, padding=1, stride=1, name="L4_C"),
            ops.ReLU(name="L4_R"),
            ops.MaxPooling(kernel_size=2, stride=2, name="L4_M"),

            # Layer 5 (B, 128, 7, 7) -> (B, 256, 7, 7)
            ops.Conv2D(in_channels=128, out_channels=256, kernel_size=3, padding=1, stride=1, name="L5_C"),
            ops.ReLU(name="L5_R"),

            # Layer 6 (B, 256, 7, 7) -> (B, 256, 7, 7)
            ops.Conv2D(in_channels=256, out_channels=256, kernel_size=3, padding=1, stride=1, name="L6_C"),
            ops.ReLU(name="L6_R")
        ]

        # Layer 7 (B, 256*7*7) -> (B, 256)
        self.fc_layers = [
            ops.FullyConnected(in_feature=256*7*7, out_feature=256, name="L7_FC"),
            ops.ReLU(name="L7_R"),

        # Layer 8 (B, 256) -> (B, 10)
            ops.FullyConnected(in_feature=256, out_feature=10, name="L8_FC")
        ]

    def backprop(self, lr, m=None) -> None:
        # Backward
        #dout = self.criterion.backward(pred, label)
        dout = self.criterion.backward()
        for i in range(len(self.fc_layers)-1, -1, -1):
            dout = self.fc_layers[i].backward(dout)
        dout = dout.reshape(dout.shape[0], 256, 7, 7)
        for i in range(len(self.conv_layers)-1, -1, -1):
            dout = self.conv_layers[i].backward(dout)
        # Update
        for layer in self.conv_layers:
            layer.update(lr, m)
        for layer in self.fc_layers:
            layer.update(lr, m)
        return dout

    def forward(self, x: np.ndarray):
        i = 0
        for layer in self.conv_layers:
            i+=1
            x = layer.forward(x) 
        x = x.reshape(x.shape[0],-1)
        for layer in self.fc_layers:
            x = layer.forward(x)
        return x

    def save(self, fileName: str):
        with open(fileName, "wb") as f:
            pickle.dump(self, f)

    def load(self, fileName: str):
        with open(fileName, "rb") as f:
            self = pickle.load(f)

In [4]:
model = VGG8()
model.load("220124_2023.pkl")

In [5]:
model.conv_layers[0].W

array([[[[4.62458459e-03, 3.99824478e-04, 9.77968962e-03],
         [3.84079777e-03, 4.77061568e-03, 5.50048784e-04],
         [2.98215712e-03, 7.95168498e-04, 8.16629981e-04]]],


       [[[3.42122452e-03, 7.39672426e-03, 7.38935714e-03],
         [7.84431277e-03, 5.22945729e-04, 7.69617609e-03],
         [6.01127653e-03, 8.66694491e-03, 6.34720227e-03]]],


       [[[7.00884866e-04, 1.57810622e-03, 4.09459854e-03],
         [8.33626264e-03, 5.42549515e-04, 4.09612732e-04],
         [2.68228777e-03, 5.30477856e-03, 9.12786787e-03]]],


       [[[1.02738764e-03, 5.46235258e-03, 5.23236548e-03],
         [4.15337921e-05, 2.61958704e-03, 6.35949423e-03],
         [8.76873453e-03, 5.33889774e-03, 4.83703923e-03]]],


       [[[1.31797691e-03, 5.26725336e-03, 6.85928678e-03],
         [5.08714967e-03, 6.14111710e-03, 1.54220454e-03],
         [7.24590486e-03, 8.70907196e-03, 4.90356682e-03]]],


       [[[2.90811340e-04, 1.90264634e-04, 6.73788771e-03],
         [9.54841162e-03, 2.20390224

In [9]:
with open("vgg8_params.bin", "wb") as f:
    for layer in model.conv_layers:
        try:
            W = layer.W.astype(np.float32)
            b = layer.b.astype(np.float32)
            f.write(W.tobytes())
            f.write(b.tobytes())
        except:
            print(f"{layer.name} not have params.")
    for layer in model.fc_layers:
        try:
            W = layer.W.astype(np.float32)
            b = layer.b.astype(np.float32)
            f.write(W.tobytes())
            f.write(b.tobytes())
        except:
            print(f"{layer.name} not have params.")

L1_R not have params.
L2_R not have params.
L2_M not have params.
L3_R not have params.
L4_R not have params.
L4_M not have params.
L5_R not have params.
L6_R not have params.
L7_R not have params.


In [12]:
model.conv_layers[0].W.astype(np.float32).flatten()[0].tobytes().hex()

'd489973b'

In [16]:
model.conv_layers[0].W.astype(np.float32).flatten()[2]

0.00977969

In [32]:
model.conv_layers[2].W.astype(np.float32).flatten()

array([0.00950201, 0.00823721, 0.00605205, ..., 0.00931372, 0.00071106,
       0.0043539 ], dtype=float32)

In [21]:
model.conv_layers